In [1]:
import cantera as ct
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

import rmgpy
from rmgpy.data.thermo import ThermoDatabase
from rmgpy.data.kinetics import KineticsDatabase
from rmgpy.molecule import Molecule
from rmgpy.species import Species
from rmgpy.reaction import Reaction
import inspect
import copy
from rmgpy.kinetics.surface import SurfaceArrhenius
from rmgpy.kinetics.surface import StickingCoefficient

import rmgpy.chemkin as Chemkin

In [2]:
# quick check that we are using the correct rmgpy and version
print('using rmgpy at: ',inspect.getfile(rmgpy))
print('using rmgpy version: ', rmgpy.__version__)

using rmgpy at:  /home/blais.ch/_02_RMG_envs/RMG_julia_env/RMG-Py/rmgpy/__init__.py
using rmgpy version:  3.1.0


In [3]:
rmg_py_path = inspect.getfile(rmgpy).split("rmgpy")[0]
rmg_db_path = rmg_py_path.split("RMG-Py")[0] + "RMG-database/"
rmg_db_path

'/home/blais.ch/_02_RMG_envs/RMG_julia_env/RMG-database/'

load files

In [4]:
park_xl_file = './park_thermo_and_rates.xlsx'
BE_sheet='Binding Energies'
rxn_sheet = 'reactions'
be_df = pd.read_excel(park_xl_file, sheet_name=BE_sheet, engine='openpyxl')
rxn_df = pd.read_excel(park_xl_file, sheet_name=rxn_sheet, engine='openpyxl')

Constants

In [5]:
site_density_mol_cm = 2.943e-09
site_density_si = site_density_mol_cm * 1e4

load thermo database

In [6]:
db_input_path = rmg_db_path + 'input/'

In [7]:
library_path = db_input_path + 'thermo/'
thermo_libraries = [
    'primaryThermoLibrary', 
    'thermo_DFT_CCSDTF12_BAC',
    'DFT_QCI_thermo',
    'surfaceThermoPt111',
    
]
thermo_database = ThermoDatabase()
thermo_database.load(
    library_path,
    libraries=thermo_libraries,
    depository=False,
    surface=True
)

import kinetic data 

In [8]:
kin_libraries_dir = db_input_path + "kinetics/libraries/Surface/"
kin_fam_dir = db_input_path + "kinetics/families/"

kinetics_libraries = [
    'CPOX_Pt/Deutschmann2006_adjusted',   
]

kinetics_families = ['surface']

kinetics_database = KineticsDatabase()
kinetics_database.load_recommended_families(kin_fam_dir  + 'recommended.py')
kinetics_database.load_families(
    path=kin_fam_dir,
    families=kinetics_families,
)

kinetics_database.load_libraries(
    kin_libraries_dir,
    libraries=kinetics_libraries
)

get thermo for molecule

We need to just manually make the right species, not a great way to translate from their structure to smiles

In [9]:
# need a dictionary translating species names to smiles
spec_smiles_dict = {
    'CO*':'O=C=[*]', 
    'CO2*':'O=C=O.[*]', 
    'H*':'[H]*',
    'H2O*':'O.[*]',
    'CH3OH*':'CO.[*]',
    'O*':'O=[*]',
    'OH*':'O[*]',
    'HCO*':'O=C*',
#     'HCOO**':'O=CO[*][*]', #formate, bidentate
    'HCOO**':'O=CO[*].[*]', # formate, bidentate, plus extra X
    'H2CO2*':'[*]OCO[*]', # its monodentate in model but dft shows bidentate
    'COOH*':'O=C(O)[*]',
    'CH2O*':'C=O.[*]',
    'CH3O*':'CO[*]',
    'CH3O2*':'OCO[*]',
    '*':'[*]',
}

# also need a dict of gas phase species to get be's from
gas_pre_dict = {
    'CO*':'[C-]#[O+]', 
    'CO2*':'O=C=O', 
    'H*':'[H]',
    'H2O*':'O',
    'CH3OH*':'CO',
    'O*':'[O]',
    'OH*':'[OH]',
    'HCO*':'[CH]=O',
    'HCOO**':'[O]C=O', #formate, bidentate
    'H2CO2*':'[O]C[O]',
    'COOH*':'O=[C]O',
    'CH2O*':'C=O',
    'CH3O*':'C[O]',
    'CH3O2*':'[O]CO',
    '*':'[*]',
}

gas_smiles_dict = {
    'CO':'[C-]#[O+]', 
    'CO2':'O=C=O', 
    'H2O':'O',
    'CH3OH':'CO',
    'CH2O':'C=O',
    'H2':'[H][H]',
}

In [10]:
def get_thermo(spec_str):
    '''
    takes a string input and returns a species object with complete thermo
    this may already exist in RMG.
    '''
    spec = Species()
    spec.from_smiles(spec_str)
    est_thermo = thermo_database.get_thermo_data(spec,metal_to_scale_to="Cu111")
    spec.thermo = est_thermo
    return spec

In [11]:
def get_gas_phase_precurs(spec):
    ''' 
    adapted from ThermoDatabase method: 
    get_thermo_data_for_surface_species()
    '''
    dummy_molecules = spec.molecule[0].get_desorbed_molecules()
    
    for mol in dummy_molecules:
        mol.clear_labeled_atoms()
    if len(dummy_molecules) == 0:
        raise RuntimeError(f"Cannot get thermo for gas-phase molecule")

    # if len(molecule) > 1, it will assume all resonance structures have already been 
    #generated when it tries to generate them, so evaluate each configuration separately 
    # and pick the lowest energy one by H298 value
    
    gas_phase_species_from_libraries = []
    gas_phase_species_estimates = []
    for dummy_molecule in dummy_molecules:
        dummy_species = Species()
        dummy_species.molecule = [dummy_molecule]
        dummy_species.generate_resonance_structures()
        dummy_species.thermo = thermo_database.get_thermo_data(dummy_species)
        if dummy_species.thermo.label:
            gas_phase_species_from_libraries.append(dummy_species)
        else:
            gas_phase_species_estimates.append(dummy_species)

    # define the comparison function to find the lowest energy
    def lowest_energy(species):
        if hasattr(species.thermo, 'H298'):
            print(species.thermo.H298.value_si)
            return species.thermo.H298.value_si
        else:
            print(species.thermo.get_enthalpy(298.0))
            return species.thermo.get_enthalpy(298.0)
        

    if gas_phase_species_from_libraries:
        species = min(gas_phase_species_from_libraries, key=lowest_energy)
    else:
        species = min(gas_phase_species_estimates, key=lowest_energy)

    thermo = species.thermo
    
    return species

generate dictionary of current binding energies

In [12]:
be_dict = {}
gas_species_dict = {}
for label in spec_smiles_dict.keys():
    surf_spec = get_thermo(spec_smiles_dict[label])
    gas_spec = get_thermo(gas_pre_dict[label])
    surf_h298 = surf_spec.thermo.get_enthalpy(298)
    gas_h298 = gas_spec.thermo.get_enthalpy(298)
    
    be_dict[label] = (surf_h298 - gas_h298)/9.6e4

In [13]:
be_dict_park = {}
for i in range(len(be_df)):
    species = be_df['Species'][i].strip()
    be_park = be_df["BE"][i]
    be_dict_park[species] = be_park
be_dict_park

{'CO*': -0.71,
 'CO2*': -0.04,
 'H*': -2.4,
 'H2O*': -0.18,
 'CH3OH*': -0.19,
 'O*': -4.89,
 'OH*': -3.13,
 'HCO*': -1.23,
 'HCOO**': -2.75,
 'H2CO2*': -3.81,
 'COOH*': -1.56,
 'CH2O*': -0.06,
 'CH3O*': -2.26,
 'CH3O2*': -2.2,
 '*': 0.0}

In [14]:
# dir(species_dict['CO*'].thermo)
# species_dict['CO*'].thermo.get_enthalpy(298)

In [15]:
species_dict = {}
for spec_name in be_df['Species']:
    smiles = spec_smiles_dict[spec_name.strip()]
    spec = get_thermo(smiles)
    spec.label = spec_name
    species_dict[spec_name.strip()] = spec
    
# manually do surface site 
# species_dict['*'] = get_thermo(spec_smiles_dict['*'])

gas_species_dict = {}
for spec_name in gas_smiles_dict.keys():
    smiles = gas_smiles_dict[spec_name.strip()]
    spec = get_thermo(smiles)
    spec.label = spec_name
    gas_species_dict[spec_name.strip()] = spec
gas_species_dict

{'CO': Species(label="CO", thermo=ThermoData(Tdata=([300,400,500,600,800,1000,1500],'K'), Cpdata=([6.95,7.02,7.12,7.26,7.58,7.86,8.35],'cal/(mol*K)'), H298=(-26.31,'kcal/mol'), S298=(47.2,'cal/(mol*K)'), Cp0=(29.1007,'J/(mol*K)'), CpInf=(37.4151,'J/(mol*K)'), label="""CO""", comment="""Thermo library: primaryThermoLibrary"""), molecule=[Molecule(smiles="[C-]#[O+]")], molecular_weight=(28.01,'amu')),
 'CO2': Species(label="CO2", thermo=ThermoData(Tdata=([300,400,500,600,800,1000,1500],'K'), Cpdata=([8.86,9.86,10.64,11.27,12.25,12.93,13.82],'cal/(mol*K)'), H298=(-93.92,'kcal/mol'), S298=(51.07,'cal/(mol*K)'), Cp0=(29.1007,'J/(mol*K)'), CpInf=(62.3585,'J/(mol*K)'), label="""CO2""", comment="""Thermo library: thermo_DFT_CCSDTF12_BAC"""), molecule=[Molecule(smiles="O=C=O")], molecular_weight=(44.0094,'amu')),
 'H2O': Species(label="H2O", thermo=ThermoData(Tdata=([300,400,500,600,800,1000,1500],'K'), Cpdata=([8.038,8.18,8.379,8.624,9.195,9.766,11.019],'cal/(mol*K)'), H298=(-57.797,'kcal/mol'

estimated thermo be

In [16]:
def update_thermo(spec, name, be1, be2):
    '''
    updates species thermo given an input for binding energy.
    input species object (spec)
    park name as string (name)
    two floats for the original binding
    energy (be1) and the "correct" binding energy (be2)
    '''
    spec_new = copy.deepcopy(spec)
    ev_2_kj = 9.6e4
    be_diff = (be_dict[name] - be_dict_park[name])*9.6e4
    new_h298 = spec.thermo.H298.value_si - be_diff
    spec_new.thermo.H298.value_si = new_h298
    print(name, id(spec_new.thermo.H298.value_si), id(spec.thermo.H298.value_si))
    print(name, spec_new.thermo.H298.value_si, spec.thermo.H298.value_si, be_diff)
    return spec_new

In [17]:
new_thermo_spec_dict = {}
for name, spec in species_dict.items():
    spec_new = update_thermo(
        spec, 
        name,
        be_dict[name], 
        be_dict_park[name],   
    )
    new_thermo_spec_dict[name] = spec_new
new_thermo_spec_dict['*'] = species_dict['*']

CO* 47883591547952 47883591547952
CO* -178241.03999999998 -181822.28766727418 -3581.2476672741946
CO2* 47883591547408 47883591547408
CO2* -396801.28 -388622.7245530643 8178.555446935702
H* 47883591547728 47883591547728
H* -12401.047999999982 -10558.930129219953 1842.1178707800295
H2O* 47883591547952 47883591547952
H2O* -259102.648 -263455.73733153223 -4353.089331532247
CH3OH* 47883591547408 47883591547408
CH3OH* -220034.31999999992 -246559.8425112978 -26525.522511297866
O* 47883591547728 47883591547728
O* -220211.67199999996 -178709.2068455112 41502.46515448876
OH* 47883591547952 47883591547952
OH* -263397.208 -174960.8073592942 88436.40064070578
HCO* 47883591547408 47883591547408
HCO* -76156.31999999999 -174562.55232281738 -98406.23232281739
HCOO** 47883591547728 47883591547728
HCOO** -391026.24 -344038.84448164486 46987.3955183551
H2CO2* 47883591547728 47883591547728
H2CO2* -300139.183266992 -360036.3922302817 -59897.20896328971
COOH* 47883591547408 47883591547408
COOH* -333019.19999

In [18]:
def convert_to_nasa(spec):
    thermo = spec.thermo
    thermo_nasa = thermo.to_nasa(298, 1500, 1000)
    spec.thermo = thermo_nasa

In [19]:
# combine gas and surface species dicts
for spec in new_thermo_spec_dict.values():
    convert_to_nasa(spec)
for spec in gas_species_dict.values():
    convert_to_nasa(spec)

combined_species_dict = {**new_thermo_spec_dict, **gas_species_dict}

In [20]:
def make_reaction(reactants, products, rxn_str, A, Ea, stick = False,):
    '''
    make a rmgpy reaction object. 
    takes a list of the species objects for 
    the reactants and products.
    takes a string for the reaction string
    if Stick is true, A-factor is the sticking coefficient
    '''
    if stick: 
        kinetics = StickingCoefficient(
            A=A, 
            n=0.0, 
            Ea=Ea, 
            T0=(1.0, "K"), 
            Tmin=None, 
            Tmax=None, 
            Pmin=None, 
            Pmax=None,
            coverage_dependence=None, 
            comment=''
        )
    else: 
        kinetics = SurfaceArrhenius(
            A=A, 
            n=0.0, 
            Ea=Ea, 
            T0=(1.0, "K"), 
            Tmin=None, 
            Tmax=None, 
            Pmin=None, 
            Pmax=None,
            coverage_dependence=None, 
            comment=''
        ) 
        
    
    # use the rmgpy reaction object 
    rxn = Reaction(
        index=-1,
        label=rxn_str,
        reactants=reactants,
        products=products,
        specific_collider=None,
        kinetics=kinetics,
        network_kinetics=None,
        reversible=True,
        transition_state=None,
        duplicate=False,
        degeneracy=1,
        pairs=None,
        allow_pdep_route=False,
        elementary_high_p=False,
        allow_max_rate_violation=False,
        rank=None,
        comment='',
        is_forward=None,
        )
    
    return rxn

In [21]:
rxn_spec_dict = {}
rxn_dict = {}
rxn_dict_coeff = {}
rxn_list = {}
for index, row in rxn_df.iterrows():
    rxn_raw = row['eqtn']
    rxn = rxn_raw.strip()
    reactants, products = rxn.split("<=>")
    reac_spl = reactants.split("+")
    prod_spl = products.split("+")
    
    # retain to list with stoichiometric coeff
    # just in case we need it
    reac_spl_coeff = reac_spl
    prod_spl_coeff = prod_spl
    
    # expand split reactant/product string so 
    # reactants with "2" as prefix become two 
    # separate strings 
    # e.g. 2OH --> OH, OH
    for reac in reac_spl:
        if reac.startswith("2"):
            reac_dup = reac.replace("2","")
            reac_spl.remove(reac)
            reac_spl.extend([reac_dup]*2)
            
    for prod in prod_spl:
        if prod.startswith("2"):
            prod_dup = prod.replace("2","")
            prod_spl.remove(prod)
            prod_spl.extend([prod_dup]*2) 
    
    rxn_dict[rxn] = [reac_spl, prod_spl] 
    rxn_dict_coeff[rxn] = [reac_spl_coeff, prod_spl_coeff] 
    
    if row['Ar'] == 'N/A' and row['stick']:
        # if no rate info and sticking coefficient
        A = 1.0
        Ea = (0,'J/mol')
    else:
        # we are making a concession here. rates that do 
        # not have an A-factor or Ea specified are quasi-
        # equilibrated, so I am setting the A-factor to the 
        # highest value (1e22 1/s) in the mechanism, and
        # making it barrierless (Ea=0 eV)

        if len(reac_spl) > 1:
            A = (float(row['Af'] / site_density_si), 'm^2/(mol*s)') # units of mol/m^2/s
        else:
            A = (float(row['Af'] / site_density_si), 's^-1') # units of mol/m^2/s
        
        Ea = (float(row['Ef (eV)'] * 9.6e4), 'J/mol') # units of J/mol
        
    
    rxn_spec_dict[rxn] = [
        [combined_species_dict[reac] for reac in reac_spl], 
        [combined_species_dict[prod] for prod in prod_spl],
    ]
    
    rxn_obj = make_reaction(
        rxn_spec_dict[rxn][0], 
        rxn_spec_dict[rxn][1], 
        rxn, 
        A, 
        Ea, 
        row['stick'],
    )
    rxn_list[rxn] = rxn_obj

In [22]:
len(rxn_list)
# rxn_list['CO+*<=>CO*'].to_chemkin()

28

In [23]:
for index, row in rxn_df.iterrows():
    print(row['Af'])   

2.5499999999999998e-20
1.75e-18
1.73e-09
1.3100000000000001e-17
0.0
5.509999999999999e-21
3.83e+16
1e+20
1e+22
1e+18
607000000000.0
8.78e+17
157000000000.0
1e+22
3.17e+18
1.94e+17
1.18e+17
7670000000000.0
1e+22
194000000000.0
51600000.0
197000000000000.0
43099999999999.99
8150000000000.0
2.08e+19
1e+22
131.0
1e+22


### write chemkin

write gas chemkin

In [24]:
# make inputs into lists for chemkin file write
chemkin_specs = []
for spec in gas_species_dict.values():
    chemkin_specs.append(spec)

chemkin_rxns = []

# can I pass an empty list for reactions? 
Chemkin.save_chemkin_file('./test_chemkin.inp', chemkin_specs, chemkin_rxns, verbose=True, check_for_duplicates=True,)

write surface chemkin

In [25]:
# make inputs into lists for chemkin file write
chemkin_specs = []
for spec in new_thermo_spec_dict.values():
    chemkin_specs.append(spec)

chemkin_rxns = []
for rxn in rxn_list.values():
    chemkin_rxns.append(rxn)
    
Chemkin.save_chemkin_surface_file('./test_chemkin_surface.inp', chemkin_specs, chemkin_rxns, verbose=True, check_for_duplicates=True,
                              surface_site_density=None)

make a cantera file from the two chemkin files

In [26]:
from cantera import ck2cti
parser = ck2cti.Parser()
parser.convertMech(
    './test_chemkin.inp', 
    outName='test_cantera.cti', 
    quiet=True, 
    permissive=True, 
    surfaceFile='./test_chemkin_surface.inp'
)

['surface1']

In [29]:
rxn_list.keys()

dict_keys(['CO+*<=>CO*', 'CO2+*<=>CO2*', 'H2+2*<=>2H*', 'H2O+*<=>H2O*', 'CH2O+*<=>CH2O*', 'CH3OH+*<=>CH3OH*', 'CO*+O*<=>CO2*+*', 'CO*+OH*<=>COOH*+*', 'CO2*+H*<=>COOH*+*', 'CO2*+H2O*<=>COOH*+OH*', 'H2O*+*<=>OH*+H*', 'OH*+*<=>O*+H*', '2OH*<=>H2O*+H*', 'CO2*+H*<=>HCOO**', 'HCOO**+H*<=>H2CO2*+2*', 'H2CO2*+H*<=>CH3O2*+*', 'H2CO2*+*<=>CH2O*+O*', 'CH3O2*+*<=>CH2O*+OH*', 'CH2O*+H*<=>CH3O*+*', 'CH3O*+H*<=>CH3OH*+*', 'CO*+H*<=>HCO*+*', 'HCOO**<=>HCO*+O*', 'HCO*+H*<=>CH2O*+*', 'CO*+OH*<=>HCO*+O*', 'CO*+H2O*<=>HCO*+OH*', 'CH3O*+CO*<=>CH2O*+*', 'CH3O*+HCO*<=>CH3OH*+CO*', 'CH3O2*+H*<=>CH2O*+H2O*'])

### edit excel file 

In [9]:
import pandas as pd 
from openpyxl import load_workbook

park_xl_file = './park_thermo_and_rates.xlsx'
#load excel file
workbook = load_workbook(filename=park_xl_file)
 
#open workbook
sheet = workbook.active
 
#modify the desired cell
cell = "C4"
print(sheet[cell].value)
 
#save the file
# workbook.save(filename=park_xl_file)
# rxn_df = pd.read_excel(park_xl_file, sheet_name=rxn_sheet, engine='openpyxl')



1e-4


### change cell value

In [10]:
sheet[cell] = '1e-5'
workbook.save(filename=park_xl_file)